In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
import requests
import pandas as pd
import time
from datetime import datetime

In [2]:
def init_browser():
    chrome_location = "C:\\Users\\HZhou\\UCSD201809DATA4\\02-Homework\\12-Web-Scraping-and-Document-Databases\\Instructions\\chromedriver.exe"
    chrome_location = chrome_location.replace("\\","/")
    executable_path = {"executable_path":chrome_location}
    
    return Browser("chrome", **executable_path, headless=False)

In [3]:
def get_soup_object(url):
    browser = init_browser()
    browser.visit(url)
    soup = BeautifulSoup(browser.html,"html.parser")
    return soup

In [4]:
def get_latest_NASA_news():
    NASA_MARS_url = "https://mars.nasa.gov/news/"
    news_soup = get_soup_object(NASA_MARS_url)
    latest_news = news_soup.find_all("div",{"class":"list_text"})[0]
    return {
        "title":latest_news.find("div",{"class":"content_title"}).text,
        "content":latest_news.find("div",{"class":"article_teaser_body"}).text
    }

In [5]:
def get_MARS_img():
    JPL_home_url = "https://www.jpl.nasa.gov"
    JPL_img_url = JPL_home_url+"/spaceimages/?search=&category=Mars"
    jpl_soup = get_soup_object(JPL_img_url)
    items = jpl_soup.find("div",{"class":"carousel_items"})
    img_title = items.find("h1",{"class":"media_feature_title"}).text
    featured_img = items.find("article")
    img_url = JPL_home_url+featured_img['style'].split(':')[1].split('\'')[1]
    return {
            "title":img_title,
            "img_url":img_url
           }

In [6]:
def get_MARS_temperature():
    twitter_report_url = "https://twitter.com/marswxreport?lang=en"
    temp_soup = get_soup_object(twitter_report_url)
    return temp_soup.find("ol",{"id":"stream-items-id"}).find("li").find("p").text

In [7]:
def get_MARS_facts():
    df = pd.read_html("https://space-facts.com/mars/")[0]
    df = df.rename(columns={0:"Description",1:"Value"})
    df = df.set_index("Description")
    return df.to_dict()['Value']

In [8]:
def scrape():
    mars_news = get_latest_NASA_news()
    mars_img = get_MARS_img()
    mars_facts = get_MARS_facts()
    mars_temp = get_MARS_temperature()
    return {
        "date":datetime.now(),
        "news":mars_news,
        "featured_img":mars_img,
        "facts":mars_facts,
        "temperature":mars_temp,
    }

In [9]:
scrape()

{'date': datetime.datetime(2019, 1, 27, 21, 48, 15, 121495),
 'news': {'title': 'Rover Team Beaming New Commands to Opportunity on Mars',
  'content': 'Engineers at JPL have begun transmitting a new set of commands to Opportunity in an attempt to compel the Mars rover to contact Earth.'},
 'featured_img': {'title': '\n\t\t\t\t  Neptune Full Disk\t\t\t\t',
  'img_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA00046-1920x1200.jpg'},
 'facts': {'Equatorial Diameter:': '6,792 km',
  'Polar Diameter:': '6,752 km',
  'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.52 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Surface Temperature:': '-153 to 20 °C',
  'First Record:': '2nd millennium BC',
  'Recorded By:': 'Egyptian astronomers'},
 'temperature': 'Sol 2299 (2019-01-24), high -5C/23F, low -74C/-101F, pressure at 8.18 hPa, daylight 06:46-18:55pic.twitter.com/UNiNdu3U8t'}